<a href="https://colab.research.google.com/github/taka-onohara/google-colab/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np



# Load data
#pwt90 = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt90.dta')
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
#data = pwt90.loc[pwt90['country'].isin(["Australia","Austria","Belgium","Canada",
                                        #"Denmark","Finland","France","Germany",
                                        #"Greece","Iceland","Ireland","Italy",
                                        #"Japan","Netherlands","New Zealand","Norway",
                                        #"Portugal","Spain","Sweden","Switzerland",
                                        #"United Kingdom","United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh','avh']]
data = pwt1001.loc[pwt1001['country'].isin(["France","Germany","Canada","Italy","Japan","United Kingdom","United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh','avh']]
#data = data.loc[(data['year'] >= 1960) & (data['year'] <= 2000)].dropna()
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Y = rgdpna, K = rkna, L = emp (* avh?), alpha = 1 - labsh then fix by country, A = (the rest)
data['Y'] = data['rgdpna']
data['K'] = data['rkna']
data['L'] = data['emp']
data['alpha'] = 1 - data['labsh']
data['alpha'] = data.groupby('countrycode')['alpha'].transform('mean')
data['A'] = data['Y'] / ((data['K'] ** data['alpha']) * (data['L'] ** (1 - data['alpha'])))

data['test'] = data['A'] * (data['K'] ** data['alpha']) * (data['L'] ** (1 - data['alpha']))
data.drop(data.columns[[2,3,4,5,6]], axis=1, inplace=True)
# print(data.head(10))

# y = Y/L, k = K/L, ln(y) = ln(A) + alpha * ln(k)
data['ln(y)'] = np.log(data['Y'] / data['L'])
data['ln(A)'] = np.log(data['A'])
data['ln(k)'] = np.log(data['K'] / data['L'])

data['test1'] = data['ln(A)'] + data['alpha'] * data['ln(k)']
#print(data.head(10))

data = data.sort_values('year')
grouped_data = data.groupby('countrycode')

data['growth_rate'] = grouped_data['ln(y)'].diff() * 100
data['TFP_growth'] = grouped_data['ln(A)'].diff() * 100
data['capital_deepening'] = data['alpha'] * (grouped_data['ln(k)'].diff() * 100)

data = data.dropna()

data['test2'] = data['TFP_growth'] + data['capital_deepening']
#print(data['alpha'].head(25))


#data.rename(columns = {'alpha':'capital_share'}, inplace = True)
#data['TFP_share'] = 1 - data['capital_share']
data['TFP_share'] = data['TFP_growth'] / data['growth_rate']
data['capital_share'] = data['capital_deepening'] / data['growth_rate']
#print(data[['TFP_share','TFP_share1','capital_share','capital_share1']].head(10))

summary = data.groupby('countrycode').agg({'growth_rate': 'mean',
                                           'TFP_growth':'mean',
                                           'capital_deepening':'mean',
                                           })
summary['TFP_share'] = summary['TFP_growth'] / summary['growth_rate']
summary['capital_share'] = summary['capital_deepening'] / summary['growth_rate']

print('---summary---')
print(summary)

overall = summary[['growth_rate','TFP_growth','capital_deepening']].mean()
overall['TFP_share'] = overall['TFP_growth'] / overall['growth_rate']
overall['capital_share'] = overall['capital_deepening'] / overall['growth_rate']
print('---overall---')
print(overall)

---summary---
             growth_rate  TFP_growth  capital_deepening  TFP_share  \
countrycode                                                          
AUS             1.724265    1.136190           0.588074   0.658942   
AUT             3.038354    1.623496           1.414857   0.534334   
BEL             2.684886    1.697809           0.987075   0.632358   
CAN             1.438622    0.822413           0.616210   0.571667   
CHE             1.391351    0.484549           0.906803   0.348258   
DEU             2.694461    1.507530           1.186930   0.559492   
DNK             2.279649    1.092271           1.187379   0.479140   
ESP             3.436415    1.838474           1.597942   0.534998   
FIN             3.324354    1.978772           1.345582   0.595235   
FRA             2.824197    1.727326           1.096870   0.611617   
GBR             2.186675    1.274831           0.911844   0.583000   
GRC             3.464603    1.349376           2.115226   0.389475   
IRL   

C:\Users\tonoh\AppData\Local\Temp\ipykernel_16136\1643018725.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['test2'] = data['TFP_growth'] + data['capital_deepening']
C:\Users\tonoh\AppData\Local\Temp\ipykernel_16136\1643018725.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['TFP_share'] = data['TFP_growth'] / data['growth_rate']
C:\Users\tonoh\AppData\Local\Temp\ipykernel_16136\1643018725.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
